In [1]:
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 KB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 7.4 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55880 sha256=047e30acfcdd4358bec4ce5ea0f67eeac00ce84c491cc5881ec7c4e54225f7be
  Stored in directory: /root/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai


In [3]:
!openai tools fine_tunes.prepare_data -f tuning_88R_data.jsonl

Analyzing...

- Your file contains 281 prompt-completion pairs
- There are 10 duplicated prompt-completion sets. These are rows: [47, 64, 165, 213, 226, 259, 266, 269, 277, 279]
- All prompts end with suffix ` \n\nLyrics:\n`. This suffix seems very long. Consider replacing with a shorter suffix, such as `\n\n###\n\n`
- All prompts start with prefix `Artist: `
- All completions end with suffix `\n###END`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 10 duplicate rows [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `tuning_88R_data_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "tuning_88R_data_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` \n\nLyrics:\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys    
path_to_module = '/content/openai_key.py'
sys.path.append(path_to_module)
from openai_key import key

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
import os
os.environ['OPENAI_API_KEY'] = key

In [15]:
!openai api fine_tunes.create -t "tuning_88R_data_prepared.jsonl" -v "tuning_88R_data_prepared.jsonl" -m 'davinci'


Upload progress: 100% 310k/310k [00:00<00:00, 288Mit/s]
Uploaded file from tuning_88R_data_prepared.jsonl: file-ZY5ZlFe5EigYoVUFVTLSdLgv
Found potentially duplicated files with name 'tuning_88R_data_prepared.jsonl', purpose 'fine-tune' and size 310199 bytes
file-ZY5ZlFe5EigYoVUFVTLSdLgv
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Upload progress: 100% 310k/310k [00:00<00:00, 390Mit/s]
Uploaded file from tuning_88R_data_prepared.jsonl: file-CQ50YGuAfGZ5wQ8T7M63gDUl
Created fine-tune: ft-ZSV4C5Q3wXnX7Q2heJeMTZEB
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-01-05 06:10:17] Created fine-tune: ft-ZSV4C5Q3wXnX7Q2heJeMTZEB
[2023-01-05 06:10:58] Fine-tune costs $11.06
[2023-01-05 06:10:58] Fine-tune enqueued. Queue number: 1
[2023-01-05 06:11:36] Fine-tune is in the queue. Queue number: 0
[2023-01-05 06:12:36] Fine-tune started
[2023-01-05 06:16:27] Completed epoch 

In [16]:
prompt_text = "Artist: Rich Brian, Joji \n\nTopic: Beef \n\nLyrics:\n"


In [20]:
import openai
openai.api_key = key
response = openai.Completion.create(
  model="davinci:ft-personal-2023-01-05-06-24-18",
  prompt=prompt_text,
  max_tokens=256,
  temperature=0.7,
  frequency_penalty=0.5,
  stop=["\n###END"]
)

In [21]:
print(prompt_text + response['choices'][0]['text'])


Artist: Rich Brian, Joji 

Topic: Beef 

Lyrics:
 
[Chorus]
I don't wanna beef, I just wanna see the faces of my enemies
When this is all over, I'm only gonna take their respect
I just wanna see their faces when they realize they've been defeated
When this is all over, I'm only gonna take their respect (Yeah)
I just wanna see their faces when they realize they've been defeated (Yeah)

[Verse ]
And I don't want a piece of you, no one does if you're not a threat (Yeah)
And we'll probably never meet 'cause you're not on my level (No)
I'ma end up on your top ten list, don't hold your breath'Cause this whole city's gettin' shook like that one time with Desiigner(Desiigner)
Even though it's childish how y'all takin' shots at me(Shots at me)
We all know that it's 'cause y'all don't have none of your own shit to talk about (About, about)
Half of y'all are broke as hell and need to stop rappin' about it (About it)
You wanna know how many shows I played without


In [22]:
with open('RichBrian-Joji-Beef.txt', 'w') as writefile:
    writefile.write(prompt_text + response['choices'][0]['text'])